In [134]:
import pandas as pd

In [135]:
import random
def random_userid(m, n):
    return random.randint(m, n)

In [136]:
from random import randrange
# from datetime import timedelta
import datetime

def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + datetime.timedelta(seconds=random_second)

In [137]:
def random_cash_direction():
    if random.random() > 0.5:
        return 'In'
    else:
        return 'Out'

In [138]:
def random_amount(max):
    return random.randint(0, max)

In [139]:
def generate_uid(rows):
    result = []
    for i in range(rows):
        result.append(random_userid(1001,1020))
    return result

d1 = datetime.datetime.strptime('1/1/2019 12:00 AM', '%m/%d/%Y %I:%M %p')
d2 = datetime.datetime.strptime('3/1/2019 12:00 AM', '%m/%d/%Y %I:%M %p')

def generate_timestamp(rows):
    result = []
    for i in range(rows):
        result.append(random_date(d1, d2))
    return result

def generate_cash_direction(rows):
    result = []
    for i in range(rows):
        result.append(random_cash_direction())
    return result

def generate_amount(rows, max):
    result = []
    for i in range(rows):
        result.append(random_amount(max))
    return result

In [140]:
rows = 100
user_id = generate_uid(rows)
timestamp = generate_timestamp(rows)
cash_direction = generate_cash_direction(rows)
amount = generate_amount(rows, 500000)

transactions = {'User_id': user_id, 'Timestamp': timestamp, 'CashDirection': cash_direction, 'Amount': amount}

In [141]:
df = pd.DataFrame(transactions)
df['CTR'] = df['Amount'].apply(lambda x: 1 if x > 10000 else 0)
df.head(20)

,User_id,Timestamp,CashDirection,Amount,CTR
0,1005,2019-01-26 22:49:39,In,172372,1
1,1008,2019-02-08 03:02:59,In,142289,1
2,1002,2019-02-12 10:04:32,Out,187333,1
3,1017,2019-01-21 22:21:27,In,318667,1
4,1002,2019-01-13 23:20:22,In,439879,1
5,1008,2019-01-11 10:51:07,In,257272,1
6,1003,2019-01-24 07:44:28,In,246173,1
7,1016,2019-01-05 19:13:54,Out,290011,1
8,1017,2019-02-27 17:28:39,Out,307179,1
9,1007,2019-01-06 20:13:16,Out,323068,1


In [142]:
df[df['Amount'] < 10000].head()

,User_id,Timestamp,CashDirection,Amount,CTR
27,1008,2019-02-11 18:51:11,Out,3415,0
74,1014,2019-01-06 09:56:43,Out,4123,0


In [159]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
# engine = create_engine('mysql://', echo=False)
# engine = create_engine('postgresql://', echo=False)

df.to_sql('transactions', con=engine)

```SQL
SELECT t1.User_id, t1.Timestamp, t2.Timestamp, DATE(t1.Timestamp, '+7 days'), SUM(t2.CTR) 
FROM transactions t1 
JOIN transactions t2 
ON t1.User_id = t2.User_id 
WHERE t2.Timestamp >= t1.Timestamp AND t2.Timestamp < DATE(t1.Timestamp, '+7 days') 
GROUP BY t1.User_id, DATE(t1.Timestamp) 
ORDER BY t1.User_id, DATE(t1.Timestamp)
```

In [160]:
stmt = "SELECT t1.User_id, t1.Timestamp, t2.Timestamp, DATE(t1.Timestamp, '+7 days'), SUM(t2.CTR) FROM transactions t1 JOIN transactions t2 ON t1.User_id = t2.User_id WHERE t2.Timestamp >= t1.Timestamp AND t2.Timestamp < DATE(t1.Timestamp, '+7 days') GROUP BY t1.User_id, DATE(t1.Timestamp) ORDER BY t1.User_id, DATE(t1.Timestamp)"

```SQL
SELECT User_id, MAX(sum_CTR)
FROM (
      SELECT t1.User_id, SUM(t2.CTR) AS sum_CTR
      FROM transactions t1
      JOIN transactions t2
      ON t1.User_id = t2.User_id
      WHERE t2.Timestamp >= t1.Timestamp AND
            t2.Timestamp < DATE(t1.Timestamp, '+7 days')
      GROUP BY t1.User_id, DATE(t1.Timestamp)
      ORDER BY t1.User_id, DATE(t1.Timestamp)
     )
GROUP BY User_id
ORDER BY 2 DESC
LIMIT 5;
```

In [161]:
# stmt = "SELECT User_id, MAX(sum_CTR) FROM (SELECT t1.User_id, SUM(t2.CTR) AS sum_CTR FROM transactions t1 JOIN transactions t2 ON t1.User_id = t2.User_id WHERE t2.Timestamp >= t1.Timestamp AND t2.Timestamp < DATE(t1.Timestamp, '+7 days') GROUP BY t1.User_id, DATE(t1.Timestamp) ORDER BY t1.User_id, DATE(t1.Timestamp)) GROUP BY User_id ORDER BY 2 DESC LIMIT 5"

In [162]:
sql_results = engine.execute(stmt).fetchall()
df_result = pd.DataFrame(sql_results)
df_result.head(50)

,0,1,2,3,4
0,1001,2019-01-05 22:50:03.000000,2019-01-05 22:50:03.000000,2019-01-12,3
1,1001,2019-01-09 15:22:34.000000,2019-01-09 15:22:34.000000,2019-01-16,2
2,1001,2019-01-11 05:34:28.000000,2019-01-11 05:34:28.000000,2019-01-18,1
3,1001,2019-01-20 05:40:42.000000,2019-01-20 05:40:42.000000,2019-01-27,1
4,1001,2019-01-29 09:58:16.000000,2019-01-29 09:58:16.000000,2019-02-05,1
5,1001,2019-02-09 07:08:25.000000,2019-02-09 07:08:25.000000,2019-02-16,2
6,1001,2019-02-13 02:00:14.000000,2019-02-13 02:00:14.000000,2019-02-20,2
7,1001,2019-02-16 01:58:33.000000,2019-02-16 01:58:33.000000,2019-02-23,1
8,1002,2019-01-04 08:53:21.000000,2019-01-04 08:53:21.000000,2019-01-11,3
9,1002,2019-01-08 10:50:14.000000,2019-01-08 10:50:14.000000,2019-01-15,3


```SQL
SELECT t1.User_id, t1.Timestamp, t2.Timestamp, DATE(t1.Timestamp, '+30 days'), SUM(t2.Amount) as income 
FROM transactions t1 
JOIN transactions t2 
ON t1.User_id = t2.User_id 
WHERE t2.Timestamp >= t1.Timestamp AND 
      t2.Timestamp < DATE(t1.Timestamp, '+30 days') AND 
      t2.CashDirection = 'In' 
GROUP BY 1, DATE(t1.Timestamp) 
ORDER BY 1, DATE(t1.Timestamp)
```

In [163]:
# stmt2 = "SELECT t1.User_id, t1.Timestamp, t2.Timestamp, DATE(t1.Timestamp, '+30 days'), SUM(t2.Amount) as income FROM transactions t1 JOIN transactions t2 ON t1.User_id = t2.User_id WHERE t2.Timestamp >= t1.Timestamp AND t2.Timestamp < DATE(t1.Timestamp, '+30 days') AND t2.CashDirection = 'In' GROUP BY 1, DATE(t1.Timestamp) ORDER BY 1, DATE(t1.Timestamp)"

```SQL
SELECT User_id, MAX(income)
FROM (
      SELECT t1.User_id, SUM(t2.Amount) as income 
      FROM transactions t1 
      JOIN transactions t2 
      ON t1.User_id = t2.User_id 
      WHERE t2.Timestamp >= t1.Timestamp AND 
            t2.Timestamp < DATE(t1.Timestamp, '+30 days') AND
            t2.CashDirection = 'In'
      GROUP BY 1, DATE(t1.Timestamp) 
      ORDER BY 1, DATE(t1.Timestamp)
     )
WHERE income > 1000000
GROUP BY User_id
ORDER BY MAX(income) DESC
LIMIT 5;
```

In [166]:
stmt2 = "SELECT User_id, MAX(income) FROM (SELECT t1.User_id, t1.Timestamp, t2.Timestamp, DATE(t1.Timestamp, '+30 days'), SUM(t2.Amount) as income FROM transactions t1 JOIN transactions t2 ON t1.User_id = t2.User_id WHERE t2.Timestamp >= t1.Timestamp AND       t2.Timestamp < DATE(t1.Timestamp, '+30 days') AND      t2.CashDirection = 'In' GROUP BY 1, DATE(t1.Timestamp) ORDER BY 1, DATE(t1.Timestamp)) WHERE income > 1000000 GROUP BY User_id ORDER BY MAX(income) DESC LIMIT 5"

In [167]:
sql_results2 = engine.execute(stmt2).fetchall()
df_result2 = pd.DataFrame(sql_results2)
df_result2.head(50)

,0,1
0,1007,1885167
1,1017,1035772
2,1008,1018929
